In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Initialization

In [31]:
data = tools.load_data(limit=800000, offset=1500000)
data = data[filters.place_type(data, 'river') & filters.for_staircase(data)]
print len(data)

Loaded 773142 answers.
22432


## PFA

### Greedy Search

In [32]:
def pfa_fun(gamma, delta):
    elo = models.EloModel()
    pfa = models.PFAModel(prior=elo, gamma=gamma, delta=delta)
    pfa_test = PerformanceTest(pfa, landscape.data)

    pfa_test.run()
    return pfa_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05}

landscape = optimize.GreedySearch(data)
landscape.search(pfa_fun, init_parameters, init_epsilons)

altitude: 0.372917222125
parameters: {'gamma': 1.5454735905929371, 'delta': -0.20981587691740108}
epsilons: {'gamma': 0.082378900212572562, 'delta': 0.0070544758970397881}


{'delta': -0.20981587691740108, 'gamma': 1.5454735905929371}

### Smart Greedy Search

In [28]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfa(number_of_iter=10)

   0.82665   -0.11048    0.41463
   0.56001    0.15651    0.41306
   0.46866    0.28621    0.41304
   0.43680    0.33872    0.41342
   0.42541    0.35865    0.41364
   0.42129    0.36604    0.41373
   0.41978    0.36875    0.41376
   0.41924    0.36975    0.41377
   0.41904    0.37011    0.41378
   0.41897    0.37024    0.41378


{'delta': 0.37024055519326116, 'gamma': 0.41896607638479794}

## PFA/E

### Greedy Search

In [18]:
def pfae_fun(gamma, delta):
    elo = models.EloModel()
    pfae = models.PFAExt(elo, gamma=gamma, delta=delta)
    pfae_test = PerformanceTest(pfae, landscape.data)

    pfae_test.run()
    return pfae_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05}

landscape = optimize.GreedySearch(data)
landscape.search(pfae_fun, init_parameters, init_epsilons)

altitude: 0.372632705518
parameters: {'gamma': 2.5357742655073756, 'delta': -0.68447489257805927}
epsilons: {'gamma': 0.082787763454942417, 'delta': 0.042252499031600339}


{'delta': -0.68447489257805927, 'gamma': 2.5357742655073756}

## PFA/G

### Greedy Search

In [33]:
def pfag_fun(gamma, delta, decay):
    elo = models.EloModel()
    pfag = models.PFAGong(prior=elo, gamma=gamma, delta=delta, decay=decay)
    pfag_test = PerformanceTest(pfag, landscape.data)

    pfag_test.run()
    return pfag_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1, 'decay': 0.09}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'decay': 0.01}

landscape = optimize.GreedySearch(data)
landscape.search(pfag_fun, init_parameters, init_epsilons)

altitude: 0.351664784387
parameters: {'delta': 0.066327543538485606, 'gamma': 2.0243509597085665, 'decay': 0.43383481549897185}
epsilons: {'delta': 0.012437721142332396, 'gamma': 0.068442698474648483, 'decay': 0.0054865243478080311}


{'decay': 0.43383481549897185,
 'delta': 0.066327543538485606,
 'gamma': 2.0243509597085665}

### Smart Greedy Search

In [50]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfag(number_of_iter=10)

   2.22006   -0.29863    0.37966
   1.85479    0.23149    0.37130
   1.58129    0.48080    0.36938
   1.40630    0.58719    0.36961
   1.29100    0.61561    0.37005
   1.21126    0.60626    0.37027
   1.15310    0.58130    0.37032
   1.10862    0.55186    0.37028
   1.07336    0.52300    0.37021
   1.04470    0.49676    0.37015


{'decay': 0.71608391039486985,
 'delta': 0.49676161027915278,
 'gamma': 1.044699976959059}

## PFA/E/T

### Greedy Search

In [3]:
def pfaet_fun(gamma, delta, a, c):
    elo = models.EloModel()
    pfaet = models.PFAExtTiming(
        elo, gamma=gamma, delta=delta,
        time_effect_fun=time_effect_fun, a=a, c=c)
    pfaet_test = PerformanceTest(pfaet, landscape.data)

    pfaet_test.run()
    return pfaet_test.results['train'].rmse

In [6]:
time_effect_fun = 'pow'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 3, 'c': 0.3}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.03}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons)

altitude: 0.360451753429
parameters: {'a': 2.6024148491925883, 'c': 0.27, 'gamma': 1.9017194430570963, 'delta': -0.75522927033349507}
epsilons: {'a': 0.097110813079835684, 'c': 0.029999999999999999, 'gamma': 0.098934411980168502, 'delta': 0.047114808754100838}


{'a': 2.6024148491925883,
 'c': 0.27,
 'delta': -0.75522927033349507,
 'gamma': 1.9017194430570963}

In [7]:
time_effect_fun = 'log'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 1.5, 'c': 0.1}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.03}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons)

altitude: 0.360465160316
parameters: {'a': 1.7896113781264669, 'c': 0.13, 'gamma': 1.9064919213249858, 'delta': -0.80551353417791893}
epsilons: {'a': 0.093531345086511147, 'c': 0.029999999999999999, 'gamma': 0.096327384837379765, 'delta': 0.047767917160314824}


{'a': 1.7896113781264669,
 'c': 0.13,
 'delta': -0.80551353417791893,
 'gamma': 1.9064919213249858}

In [4]:
time_effect_fun = 'exp'

init_parameters = {'gamma': 2, 'delta': -1, 'a': 1, 'c': 0.01}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'a': 0.1, 'c': 0.005}

landscape = optimize.GreedySearch(data)
landscape.search(pfaet_fun, init_parameters, init_epsilons)

altitude: 0.361029441303
parameters: {'a': 1.0046999532781646, 'c': 0.01, 'gamma': 2.0061720998301338, 'delta': -0.75657801487834175}
epsilons: {'a': 0.090306802141485509, 'c': 0.0050000000000000001, 'gamma': 0.086965599971364368, 'delta': 0.042243370476929394}


{'a': 1.0046999532781646,
 'c': 0.01,
 'delta': -0.75657801487834175,
 'gamma': 2.0061720998301338}

## PFA/G/T

### Greedy Search

In [10]:
def pfagt_fun(a, c, b, d):
    elo = models.EloModel()
    pfagt = models.PFATiming(
        elo,
        time_effect_good=time_effect_fun, a=a, c=c,
        time_effect_bad=time_effect_fun, b=b, d=d)
    pfagt_test = PerformanceTest(pfagt, landscape.data)

    pfagt_test.run()
    return pfagt_test.get_results().rmse

In [34]:
time_effect_fun = 'pow'

init_parameters = {'a': 3, 'c': 0.3, 'b': 4, 'd': 0.4}
init_epsilons = {'a': 0.1, 'c': 0.03, 'b': 0.1, 'd': 0.04}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=20)

altitude: 0.348210453123
parameters: {'a': 3.1964403426577523, 'c': 0.20989502680946473, 'b': 3.7053035513700547, 'd': 0.39499211887026081}
epsilons: {'a': 0.086775859635688943, 'c': 0.013374682934615205, 'b': 0.098460728218683832, 'd': 0.035674104669819608}


{'a': 3.1964403426577523,
 'b': 3.7053035513700547,
 'c': 0.20989502680946473,
 'd': 0.39499211887026081}

In [12]:
time_effect_fun = 'log'

init_parameters = {'a': 1.7, 'c': 0.1, 'b': 0.9, 'd': 0.1}
init_epsilons = {'a': 0.05, 'c': 0.005, 'b': 0.05, 'd': 0.005}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=20)

altitude: 0.351252971311
parameters: {'a': 1.65, 'c': 0.1, 'b': 0.80003890949257284, 'd': 0.085034792796917003}
epsilons: {'a': 0.049715719406419015, 'c': 0.0050000000000000001, 'b': 0.049940081212585322, 'd': 0.0049714050763154218}


{'a': 1.65, 'b': 0.80003890949257284, 'c': 0.1, 'd': 0.085034792796917003}

In [36]:
time_effect_fun = 'exp'

init_parameters = {'a': 1.5, 'c': 0.01, 'b': 1.3, 'd': 0.01}
init_epsilons = {'a': 0.1, 'c': 0.003, 'b': 0.1, 'd': 0.003}

landscape = optimize.GreedySearch(data)
landscape.search(pfagt_fun, init_parameters, init_epsilons, maxiter=20)

### Smart Greedy Search

In [49]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfagt(number_of_iter=10)

   3.51309   -0.25624    0.37776
   3.68964    0.04912    0.37706
   3.46514    0.04062    0.37575
   3.25846    0.03920    0.37536
   3.05692    0.03611    0.37501
   2.86231    0.03415    0.37464
   2.67456    0.03202    0.37430
   2.49209    0.02844    0.37398
   2.32170    0.02788    0.37366
   2.15652    0.02566    0.37334


{'a': 2.9700846520609057,
 'c': 0.25924650629428969,
 'delta': 0.025657149029072922,
 'gamma': 2.1565236199060083}

In [3]:
landscape = optimize.GreedySearch2(data)
landscape.search_pfagt(number_of_iter=10, time_effect_fun='exp')

   2.01599    0.10362    0.38777
   1.68531    0.05819    0.38661
   1.34697    0.03985    0.38363
   1.06414    0.03902    0.38051
   0.79137    0.02831    0.37611
   0.55974    0.01953    0.37206
   0.34258    0.02003    0.36915
   0.20341    0.04402    0.37436
   0.11545    0.02245    0.38869
   0.08667    0.01565    0.39558


{'a': 3.2731618029154732,
 'c': -0.099687077060505391,
 'delta': 0.015650741856747617,
 'gamma': 0.086671413614965359}

## PFA/E/RT

In [38]:
def pfart_fun(gamma, delta, zeta):
    elo = models.EloModel()
    pfart = models.PFAResponseTime(elo, gamma=gamma, delta=delta, zeta=zeta)
    pfart_test = PerformanceTest(pfart, landscape.data)

    pfart_test.run()
    return pfart_test.get_results().rmse

init_parameters = {'gamma': 2, 'delta': -1, 'zeta': 1.5}
init_epsilons = {'gamma': 0.1, 'delta': 0.05, 'zeta': 0.1}

landscape = optimize.GreedySearch(data)
landscape.search(pfart_fun, init_parameters, init_epsilons, altitude_change=100)